In [11]:
%load_ext autoreload
%autoreload 2
from get_data import *
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
train_data_15m = get_data(
    "2021-11-01",
    "2021-12-31",
    "BTCUSDT",
    "15m"
)

train_data_1h = get_data(
    "2021-10-01",
    "2021-12-31",
    "BTCUSDT",
    "1h"
)

train_data_12h = get_data(
    "2021-01-01",
    "2021-12-31",
    "BTCUSDT",
    "12h"
)

train_data_1d = get_data(
    "2020-01-01",
    "2021-12-31",
    "BTCUSDT",
    "1d"
)

  3%|▎         | 70/2184 [00:00<00:06, 345.90it/s]

Preparing data from database


100%|██████████| 2184/2184 [00:06<00:00, 314.41it/s]


In [28]:
def grid_search(
    forecaster,
    data,
    col,
    ohlcv_size,
):
    # Lags used as predictors
    lags_grid = [6, 12, 18]

    # Regressor's hyperparameters
    param_grid = {
        'n_estimators': [100, 250, 500],
        'max_depth': [4, 6, 10]
    }

    print(f"RESULT FOR column = {col} with size = {ohlcv_size} with regressor {forecaster}")
    return grid_search_forecaster(
        forecaster=forecaster,
        y=data[col],
        param_grid=param_grid,
        lags_grid=lags_grid,
        steps=1,
        refit=False,
        metric='mean_squared_error',
        initial_train_size=int(len(data)*0.5),
        fixed_train_size=False,
        return_best=True,
        verbose=False
    )

In [29]:
# HYPER PARAMETER TUNINNG CATBOOST 15m low
result = grid_search(
    forecaster=ForecasterAutoreg(regressor=CatBoostRegressor(random_state=123, silent=True), lags=1),
    data=train_data_1h,
    col="low",
    ohlcv_size="1h"
)
result

loop lags_grid:   0%|                                               | 0/3 [00:00<?, ?it/s]

RESULT FOR column = low with size = 1h with regressor ================= 
ForecasterAutoreg 
Regressor: <catboost.core.CatBoostRegressor object at 0x0000023602869A90> 
Lags: [1] 
Window size: 1 
Included exogenous: False 
Type of exogenous variable: None 
Exogenous variables names: None 
Training range: None 
Training index type: None 
Training index frequency: None 
Regressor parameters: {'loss_function': 'RMSE', 'silent': True, 'random_state': 123} 
Creation date: 2022-04-08 04:40:40 
Last fit date: None 
Skforecast version: 0.4.3 

Number of models compared: 27


loop lags_grid: 100%|███████████████████████████████████████| 3/3 [01:39<00:00, 33.23s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3 4 5 6] 
  Parameters: {'max_depth': 6, 'n_estimators': 500}
  Backtesting metric: 449687.0502730848



,lags,params,metric,max_depth,n_estimators
5,"[1, 2, 3, 4, 5, 6]","{'max_depth': 6, 'n_estimators': 500}",4.496871e+05,6,500
4,"[1, 2, 3, 4, 5, 6]","{'max_depth': 6, 'n_estimators': 250}",4.504990e+05,6,250
8,"[1, 2, 3, 4, 5, 6]","{'max_depth': 10, 'n_estimators': 500}",4.550332e+05,10,500
7,"[1, 2, 3, 4, 5, 6]","{'max_depth': 10, 'n_estimators': 250}",4.874440e+05,10,250
2,"[1, 2, 3, 4, 5, 6]","{'max_depth': 4, 'n_estimators': 500}",5.043405e+05,4,500
6,"[1, 2, 3, 4, 5, 6]","{'max_depth': 10, 'n_estimators': 100}",5.240509e+05,10,100
1,"[1, 2, 3, 4, 5, 6]","{'max_depth': 4, 'n_estimators': 250}",5.287987e+05,4,250
3,"[1, 2, 3, 4, 5, 6]","{'max_depth': 6, 'n_estimators': 100}",5.419821e+05,6,100
0,"[1, 2, 3, 4, 5, 6]","{'max_depth': 4, 'n_estimators': 100}",5.876649e+05,4,100
10,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","{'max_depth': 4, 'n_estimators': 250}",6.948968e+05,4,250


In [14]:
# HYPER PARAMETER TUNINNG CATBOOST 15m low
result = grid_search(
    forecaster=ForecasterAutoreg(regressor=CatBoostRegressor(random_state=123, silent=True), lags=1),
    data=train_data_15m,
    col="low",
    ohlcv_size="15m"
)
result

loop lags_grid:   0%|                                               | 0/3 [00:00<?, ?it/s]

RESULT FOR column = low with size = 15m with regressor ================= 
ForecasterAutoreg 
Regressor: <catboost.core.CatBoostRegressor object at 0x0000023604B9AC40> 
Lags: [1] 
Window size: 1 
Included exogenous: False 
Type of exogenous variable: None 
Exogenous variables names: None 
Training range: None 
Training index type: None 
Training index frequency: None 
Regressor parameters: {'loss_function': 'RMSE', 'silent': True, 'random_state': 123} 
Creation date: 2022-04-07 23:34:58 
Last fit date: None 
Skforecast version: 0.4.3 

Number of models compared: 27


loop lags_grid: 100%|██████████████████████████████████████| 3/3 [13:35<00:00, 271.69s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10] 
  Parameters: {'max_depth': 10, 'n_estimators': 500}
  Backtesting metric: 228878.5455045463



,lags,params,metric,max_depth,n_estimators
8,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","{'max_depth': 10, 'n_estimators': 500}",228878.545505,10,500
7,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","{'max_depth': 10, 'n_estimators': 250}",230808.008911,10,250
5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","{'max_depth': 5, 'n_estimators': 500}",238599.053313,5,500
6,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","{'max_depth': 10, 'n_estimators': 100}",240367.951888,10,100
14,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'max_depth': 5, 'n_estimators': 500}",240880.864747,5,500
3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","{'max_depth': 5, 'n_estimators': 100}",241218.714601,5,100
17,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'max_depth': 10, 'n_estimators': 500}",241541.267283,10,500
26,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'max_depth': 10, 'n_estimators': 500}",242423.707039,10,500
16,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'max_depth': 10, 'n_estimators': 250}",242926.888433,10,250
13,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'max_depth': 5, 'n_estimators': 250}",244378.468600,5,250
